In this noebook we are going to Pre-Process our text data. Our aim after running this note is to create a csv file which has the following attributes - AuthorName, Folder, File, Message ID, Processed_Text, Raw Text. As such we are removing any unwanted data from our raw_text data using Regular Expression. In the Processed_text we are only interested in the chat(text data) sent over email and are not concered with other other data like the Filename or forwareded by etc.  

In [ ]:
# importing libraries
import re
import numpy as np
import os
import pandas as pd

In [4]:
# this function receives raw text and removes the following items - Filename and Forwarded By data

def raw_text_processing(TextRaw):
    
    if re.search(r"X-FileName:.+\n+((.+\n\n?)+)", TextRaw):
        text = re.search(r"X-FileName:.+\n+((?:.+\n\n?)+)", TextRaw).group(1)
        
        if re.match(r"-{3,} Forwarded by.+\d+\/\d+\/\d+", text):
            return np.NaN
        
        if re.search(r"-{3,} Forwarded by.+\d+\/\d+\/\d+[\w\W]+", text):
            text = re.sub(r"-{3,} Forwarded by.+\d+\/\d+\/\d+[\w\W]+", "", text)
        
        if re.search(r"-{5,}Original Message-{5,}[\w\W].*", text):
            text = re.sub(r"-{5,}Original Message-{5,}[\w\W]*[\n]*.*", "", text)
            
        if text:
            return text
        else:
            return np.NaN
        
    elif re.search(r"X-FileName:.+\n+(.+)", TextRaw):
        text = re.search(r"X-FileName:.+\n+(.+)", TextRaw).group(1)
        
        if re.match(r"-{3,} Forwarded by.+\d+\/\d+\/\d+", text):
            return np.NaN
    
        if re.search(r"-{3,} Forwarded by.+\d+\/\d+\/\d+[\w\W]+", text):
            text = re.sub(r"-{3,} Forwarded by.+\d+\/\d+\/\d+[\w\W]+", "", text)
        
        if re.search(r"-{5,}Original Message-{5,}[\w\W].*", text):
            text = re.sub(r"-{5,}Original Message-{5,}[\w\W]*[\n]*.*", "", text)
            
        if text:
            return text
        else:
            return np.NaN
    
    else:
        return np.NaN
    

In [5]:
# this function receives the email file and returns the follwoing - EmailID, Sender, processed_text and raw text

def read_email(File):
  
    with open(File, mode="r") as f:
        try:
            TextRaw = f.read()
        except FileNotFoundError as e:
            return None, None, None, None
    if re.search(r"^Message-ID: <(\d+\.\d+\.)JavaMail\.evans@thyme>", TextRaw):
        EmailID = re.search(r"^Message-ID: <(\d+\.\d+\.)JavaMail\.evans@thyme>", TextRaw).group(1)
    else:
        raise Exception("No ID found {}".format(File))
    if re.search(r"X-From: (.+) <?", TextRaw):
        SENDER = re.search(r"X-From: (.+) <?", TextRaw).group(1)
    else:
        print("No ID fpund{}".format(File))
        SENDER = np.NaN
    
    text = raw_text_processing(TextRaw)

    return EmailID, SENDER, text, TextRaw

In [7]:
# this function receives a author folder path and extratcs all his/her emails from sent, sent_mail and sent_items sub directory

def extract_email(AUThor):
  
    RootFolder = "./maildir/"
    Folder = ["/sent/", "/_sent_mail/", "/sent_items/"]
    DataExtract = []

    for folders in Folder:
        try:
            for message in os.listdir(RootFolder + AUThor + folders):
                if os.path.isfile(RootFolder + AUThor + folders + message):
                    EmailID, SENDER, text, TextRaw = read_email(RootFolder + AUThor + folders + message)
                    if EmailID != None:
                        DataExtract.append([SENDER, AUThor, message, EmailID, text, TextRaw])
        except FileNotFoundError as e:
            continue
    return DataExtract

In [6]:
# this is our main calller function which loops over all the authors in the maildir folder

AUTHORS = os.listdir("./maildir")
number_author_folders = len(AUTHORS)
df = pd.DataFrame(columns=["Author", "Folder", "File", "Message ID", "Text", "Raw Text", "Email Folder"])

for AUThor in AUTHORS:
    EMAIL = extract_email(AUThor)
    if EMAIL:
        row = pd.DataFrame(EMAIL, columns=["Author", "Folder", "File", "Message ID", "Text", "Raw Text"])
        df = pd.concat(df, row)

df = df.drop_duplicates(["Message ID"])
df = df[df["Text"].notna()]

print(df.value_counts(["Folder"])[:20])
df

C:\Users\neels\AppData\Local\Temp\ipykernel_1748\2895039270.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.DataFrame(EMAIL, columns=["Author", "Folder", "File", "Message ID", "Text", "Raw Text"]))
C:\Users\neels\AppData\Local\Temp\ipykernel_1748\2895039270.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.DataFrame(EMAIL, columns=["Author", "Folder", "File", "Message ID", "Text", "Raw Text"]))
C:\Users\neels\AppData\Local\Temp\ipykernel_1748\2895039270.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.DataFrame(EMAIL, columns=["Author", "Folder", "File", "Message ID", "Text", "Raw Text"]))
C:\Users\neels\AppData\Local\Temp\ipykernel_1748\2895039270.py:8: FutureW

Message ID not found for File ./maildir/brawner-s/sent_items/7


C:\Users\neels\AppData\Local\Temp\ipykernel_1748\2895039270.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.DataFrame(EMAIL, columns=["Author", "Folder", "File", "Message ID", "Text", "Raw Text"]))
C:\Users\neels\AppData\Local\Temp\ipykernel_1748\2895039270.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.DataFrame(EMAIL, columns=["Author", "Folder", "File", "Message ID", "Text", "Raw Text"]))
C:\Users\neels\AppData\Local\Temp\ipykernel_1748\2895039270.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.DataFrame(EMAIL, columns=["Author", "Folder", "File", "Message ID", "Text", "Raw Text"]))
C:\Users\neels\AppData\Local\Temp\ipykernel_1748\2895039270.py:8: FutureW

Message ID not found for File ./maildir/forney-j/sent_items/158


C:\Users\neels\AppData\Local\Temp\ipykernel_1748\2895039270.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.DataFrame(EMAIL, columns=["Author", "Folder", "File", "Message ID", "Text", "Raw Text"]))
C:\Users\neels\AppData\Local\Temp\ipykernel_1748\2895039270.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.DataFrame(EMAIL, columns=["Author", "Folder", "File", "Message ID", "Text", "Raw Text"]))
C:\Users\neels\AppData\Local\Temp\ipykernel_1748\2895039270.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.DataFrame(EMAIL, columns=["Author", "Folder", "File", "Message ID", "Text", "Raw Text"]))
C:\Users\neels\AppData\Local\Temp\ipykernel_1748\2895039270.py:8: FutureW

...

C:\Users\neels\AppData\Local\Temp\ipykernel_1748\2895039270.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.DataFrame(EMAIL, columns=["Author", "Folder", "File", "Message ID", "Text", "Raw Text"]))
C:\Users\neels\AppData\Local\Temp\ipykernel_1748\2895039270.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.DataFrame(EMAIL, columns=["Author", "Folder", "File", "Message ID", "Text", "Raw Text"]))
C:\Users\neels\AppData\Local\Temp\ipykernel_1748\2895039270.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.DataFrame(EMAIL, columns=["Author", "Folder", "File", "Message ID", "Text", "Raw Text"]))
C:\Users\neels\AppData\Local\Temp\ipykernel_1748\2895039270.py:8: FutureW

Message ID not found for File ./maildir/ybarbo-p/sent_items/34


C:\Users\neels\AppData\Local\Temp\ipykernel_1748\2895039270.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.DataFrame(EMAIL, columns=["Author", "Folder", "File", "Message ID", "Text", "Raw Text"]))
C:\Users\neels\AppData\Local\Temp\ipykernel_1748\2895039270.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.DataFrame(EMAIL, columns=["Author", "Folder", "File", "Message ID", "Text", "Raw Text"]))
C:\Users\neels\AppData\Local\Temp\ipykernel_1748\2895039270.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.DataFrame(EMAIL, columns=["Author", "Folder", "File", "Message ID", "Text", "Raw Text"]))


Folder       
mann-k           8177
kaminski-v       6376
dasovich-j       4827
germany-c        4652
shackleton-s     4075
jones-t          3376
bass-e           2703
beck-s           2631
symes-k          2617
lenhart-m        2572
scott-s          2444
perlingiere-d    2339
taylor-m         2185
arnold-j         2174
love-p           2052
nemec-g          2039
fossum-d         2038
lavorato-j       1765
rogers-b         1723
delainey-d       1662
dtype: int64


,Author,Folder,File,Message ID,Text,Raw Text,Email Folder
0,Phillip K,allen-p,1,18782981.1075855378110.,Here is our forecast\n\n,Message-ID: <18782981.1075855378110.JavaMail.e...,NaN
1,Phillip K,allen-p,10,15464986.1075855378456.,Traveling to have a business meeting takes the...,Message-ID: <15464986.1075855378456.JavaMail.e...,NaN
2,Phillip K,allen-p,100,24216240.1075855687451.,test successful. way to go!!!,Message-ID: <24216240.1075855687451.JavaMail.e...,NaN
3,Phillip K,allen-p,1000,13505866.1075863688222.,"Randy,\n\n Can you send me a schedule of the s...",Message-ID: <13505866.1075863688222.JavaMail.e...,NaN
4,Phillip K,allen-p,1001,30922949.1075863688243.,Let's shoot for Tuesday at 11:45.,Message-ID: <30922949.1075863688243.JavaMail.e...,NaN
...,...,...,...,...,...,...,...
333,"Zufferli, John",zufferli-j,95,26807948.1075842029936.,This is a trade with OIL-SPEC-HEDGE-NG (John L...,Message-ID: <26807948.1075842029936.JavaMail.e...,NaN
334,"Zufferli, John",zufferli-j,96,25835861.1075842029959.,Some of my position is with the Alberta Term b...,Message-ID: <25835861.1075842029959.JavaMail.e...,NaN
335,"Zufferli, John",zufferli-j,97,28979867.1075842029988.,2\n\n,Message-ID: <28979867.1075842029988.JavaMail.e...,NaN
336,"Zufferli, John",zufferli-j,98,22052556.1075842030013.,Analyst\t\t\t\t\tRank\n\nStephane Brodeur\t\t\...,Message-ID: <22052556.1075842030013.JavaMail.e...,NaN


In [7]:
# converting dataframe to csv so that we don't need to run these steps every time 

df.to_csv("./enron.csv")


In [8]:
# this completes our Step 2